In [2]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\pnaghs1\AppData\Local\anaconda\envs\LLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.7.0+cu118)
    Python  3.10.11 (you have 3.10.17)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\pnaghs1\AppData\Local\anaconda\envs\LLM\lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3050 OEM. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Load a PDF from local or web
from pdfminer.high_level import extract_text
pdf_path = "d2l-en.pdf"  # change as needed
text = extract_text(pdf_path)


Cannot set gray non-stroke color because /'p6' is an invalid float value
Cannot set gray non-stroke color because /'p7' is an invalid float value
Cannot set gray non-stroke color because /'p8' is an invalid float value
Cannot set gray non-stroke color because /'p9' is an invalid float value
Cannot set gray non-stroke color because /'p10' is an invalid float value
Cannot set gray non-stroke color because /'p11' is an invalid float value
Cannot set gray non-stroke color because /'p12' is an invalid float value
Cannot set gray non-stroke color because /'p13' is an invalid float value
Cannot set gray non-stroke color because /'p14' is an invalid float value
Cannot set gray non-stroke color because /'p15' is an invalid float value
Cannot set gray non-stroke color because /'p17' is an invalid float value
Cannot set gray non-stroke color because /'p6' is an invalid float value
Cannot set gray non-stroke color because /'p7' is an invalid float value
Cannot set gray non-stroke color because /'p

In [5]:

# 3. Split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

# 4. Embed chunks into a Vectorstore
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

embedding = GPT4AllEmbeddings()
vectordb = Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory="rag_store")

retriever = vectordb.as_retriever(search_kwargs={"k": 3})


In [6]:

# 5. Ask a question and retrieve context
query = "What is the code for AlexNet controls the model complexity ?"
docs = retriever.get_relevant_documents(query)

# 6. Build final prompt
context = "\n\n".join([doc.page_content for doc in docs])
final_prompt = f"""Answer the following question using the context below:
Context:
{context}

Question:
{query}
"""

C:\Users\pnaghs1\AppData\Local\Temp\ipykernel_50556\1729965260.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [8]:

# 7. Run the model
inputs = tokenizer(final_prompt, return_tensors="pt").to(device)
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=800,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.1,
    )

# 8. Decode answer
answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Answer:", answer)

Answer: Answer the following question using the context below:
Context:
Although it seems that there are only a few more lines in AlexNet’s implementation than
in LeNet’s, it took the academic community many years to embrace this conceptual change
and take advantage of its excellent experimental results. This was also due to the lack of
efficient computational tools. At the time neither DistBelief (Dean et al., 2012) nor Caffe
(Jia et al., 2014) existed, and Theano (Bergstra et al., 2010) still lacked many distinguishing

Capacity Control and Preprocessing

AlexNet controls the model complexity of the fully connected layer by dropout (Section
5.6), while LeNet only uses weight decay. To augment the data even further, the training
loop of AlexNet added a great deal of image augmentation, such as flipping, clipping, and
color changes. This makes the model more robust and the larger sample size effectively
reduces overfitting. See Buslaev et al. (2020) for an in-depth review of such prepr